In [2]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

Built-in Tool - DuckDuckGo Search

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("Top news in India today.")
print(result)

2 days ago · Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at NDTV.com. 12 hours ago · Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. Feb 7, 2025 · Get Top News of the day with latest top news headlines of today. Get updated with current top news stories from India and the world only on Zee News. 1 day ago · Breaking News LIVE: PM Modi's Big Cheer For West Indies Cricket Team In Trinidad and Tobago - VIDEO Breaking News Live: Follow this page for the latest and fastest updates on … 1 day ago · Latest news headlines from India & around the world. Check out today’s news coverage live with videos & photos on NDTV.com.


In [4]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


Built-in Tool - Shell Tool

In [5]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()
results = shell_tool.invoke("ls")
print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Custom Tools

In [6]:
from langchain_core.tools import tool

In [7]:
# Step 1 - Create a function
def muliply(a,b):
  """Multiply two numbers"""
  return a*b


In [8]:
# Step 2 - Add type hints
def muliply(a:int,b:int)->int:
  """Multiply two numbers"""
  return a*b

In [9]:
# Step 3 - Add tool decorator
@tool
def multiply(a:int,b:int)->int:
  """Multiply two numbers"""
  return a*b

In [10]:
result = multiply.invoke({"a":3,"b":5})

In [11]:
print(result)

15


In [12]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [13]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


Method 2 - Using StructuredTool

In [14]:
from langchain.tools import StructuredTool
from pydantic import BaseModel,Field

In [15]:
class MultiplyInput(BaseModel):
  a : int = Field(required = True,description = "The first number to add")
  b : int = Field(required = True,description = "The second number to add")


In [16]:
def multiply_func(a:int,b:int)->int:
  return a*b


In [17]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name='multiply',
    description='Multiply Two numbers',
    args_schema = MultiplyInput
)

In [21]:
result = multiply_tool.invoke({'a':3,'b':3})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply Two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}
9
multiply
Multiply Two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


Method 3 - Using BaseTool Class

In [19]:
from langchain.tools import BaseTool
from typing import Type

In [22]:
# Arg schema using Pydantic
class MultiplyInput(BaseModel):
  a: int = Field(required=True, description="The first number to add")
  b: int = Field(required=True, description="The second number to add")

In [23]:
class MultiplyTool(BaseTool):
  name : str = 'multiply'
  description: str='Multiply Two numbers'
  args_schema:Type[BaseModel] = MultiplyInput
  def _run(self,a:int,b:int)->int:
    return a*b

In [24]:
multiply_tool = MultiplyTool()

In [25]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply Two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


Toolkit

In [27]:
from langchain_core.tools import tool
# Custom tools
@tool
def add(a:int,b:int)->int:
  """Add two numbers"""
  return a + b
@tool
def multiply(a:int,b:int)->int:
  """Multiply two numbers"""
  return a*b

In [31]:
class MathToolkit:
  def get_tools(self):
    return [add, multiply]

In [32]:
toolkit = MathToolkit()
tools = toolkit.get_tools()
for tool in tools:
  print(tool.name,"=>",tool.description)

add => Add two numbers
multiply => Multiply two numbers
